In [ ]:
import re
import time
import json
import pandas as pd
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

from langchain.vectorstores.base import VectorStoreRetriever
from langchain_community.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from geoloc_utils.ragGeocoder import get_coordinates_array, get_distances

from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv

from langchain.memory import ChatMessageHistory

from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

--------------------------------

### Создание ретриверов и определение функций

In [2]:
places_paths = ["../production_data/production_hotels_districts.csv", "../production_data/production_cafes_districts.csv", "../production_data/production_ent_districts.csv"]
docs = []

def docs_cleaning(docs):
    for i in range (len(docs)):
        cleaned_content = re.sub(r'(: [0-9+]\n0:)', ' ', docs[i].page_content).strip()
        docs[i].page_content = cleaned_content
        docs[i].page_content = docs[i].page_content.replace("description: ", "")
    return docs

def create_loader(file_path):
    loader = CSVLoader(file_path=file_path, content_columns=['description'],
        csv_args={
        'delimiter': ',',
        'quotechar': '"',
    })
    docs = loader.load()
    docs = docs_cleaning(docs)
    return docs

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def make_retriver(name: str, documents: list, embedding_generator,text_splitter, search_type: str = "similarity", search_kwargs: dict = {"k": 5}):
    text_splits = text_splitter.split_documents(documents)
    vector_store = Chroma.from_documents(collection_name=name, documents=text_splits, embedding=embedding_generator, persist_directory="/tmp/chroma_db")
    return vector_store.as_retriever(search_type=search_type, search_kwargs=search_kwargs)

hotels_docs = create_loader(places_paths[0])
cafes_docs = create_loader(places_paths[1])
ent_docs = create_loader(places_paths[2])

In [3]:
'закрыто' in ent_docs

False

In [4]:
# разделяем текст на части
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=700, chunk_overlap=100, add_start_index=True
    )
splits = text_splitter.split_documents(docs)

# создаем эмбеддинги для частей текста
embeddings_generator = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={'device': 'cuda'},
)


# создаем объект для поиска по векторному хранилищу
hotels_retriever = make_retriver("hotels", hotels_docs, embedding_generator=embeddings_generator, text_splitter=text_splitter)

ent_retriever = make_retriver("ents", ent_docs, embedding_generator=embeddings_generator, text_splitter=text_splitter)

cafes_retriever = make_retriver("cafes", cafes_docs, embedding_generator=embeddings_generator, text_splitter=text_splitter)

In [5]:
'''
if hasattr(hotels_retriever, 'vectorstore'):
    hotels_retriever.vectorstore.delete_collection()
if hasattr(ent_retriever, 'vectorstore'):
    ent_retriever.vectorstore.delete_collection()
if hasattr(cafes_retriever, 'vectorstore'):
    cafes_retriever.vectorstore.delete_collection()'''
    


"\nif hasattr(hotels_retriever, 'vectorstore'):\n    hotels_retriever.vectorstore.delete_collection()\nif hasattr(ent_retriever, 'vectorstore'):\n    ent_retriever.vectorstore.delete_collection()\nif hasattr(cafes_retriever, 'vectorstore'):\n    cafes_retriever.vectorstore.delete_collection()"

In [5]:
search_text = "а где план?"
if (("маршрут" not in search_text) and ("план" not in search_text) and ("поездк" not in search_text) and ("путь" not in search_text) and ("пути" not in search_text) and ("путешеств" not in search_text) and ("рядом" not in search_text) and ("недалеко" not in search_text) and ("расст" not in search_text)) or False:
    print(1)
else:
    print(0)

0


### RAG-конвейер

In [29]:
# используемая модель
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature = 0.7,
    api_key=API_KEY,
    base_url=PROXY_URL,
)

# промпты
template_main = """Вы являетесь дружелюбным помощником при составлении маршрутов по Новосибирской области, но можете немного и поговорить о жизни.
Используйте информацию только из найденного контекста, чтобы ответить на вопрос. При вопросе об определенном городе предоставляйте места только из него. Указывайте название, адрес, краткое описание (если оно есть) и цену мест. Не добавляйте свои данные. Отвечайте на русском, понятно, кратко и лаконично. Используйте слова из запроса пользователя.
При составлении маршрута на 1 день и более ОБЯЗАТЕЛЬНО указывайте места, где можно покушать и где переночевать. Указывай одно место для ночлега на все дни, если не просят обратного. Если попросят порекомендовать что-то рядом, то выбирайте места, которые находятся В ОДНОМ РАЙОНЕ, если мест в одном районе нет или вам неизвестен район - предупреждайте, что место не рядом 
Данные о развлечениях и санаториях Новосибирска: {context_ent}  
Данные об отелях и санаториях Новосибирска: {context_hotels}
Данные о местах питания в Новосибирске: {context_cafes}
Учитывайте историю диалога и старайтесь не допускать повтора мест при повторной просьбе от пользователя.
Если вы не знаете ответа или у вас нет данных для корректного ответа на него, просто скажите, что не знаете. 
Вопрос от пользователя: {question} 
"""
prompt = ChatPromptTemplate.from_template(template_main)

template_paths = """Если текст от бота не является маршрутом или планом путешествия, то ВЕРНИ ИСХОДНЫЙ ТЕКСТ. Иначе, если текст от бота содержит в себе список различных мест, то добавь в текст информацию о расстояниях между теми объектами, которые есть в тексте от бота. Напишите примерное время в пути при передвижении пешком и на машине. Предупредите, что расстояния и время приблизительные и необходимо уточнить информацию в интернете. В ответе оставь только текст для пользователя.
Текст от бота: {prompt}  
Данные о расстояниях между объектами: {distances}
"""
prompt_paths = ChatPromptTemplate.from_template(template_paths)

In [ ]:
def get_retriver_categories(llm, question):
    template = """Есть вопрос от пользователя: {question} 
    Оцени, будет ли пользователю полезна для ответа на вопрос информация о: местах проживания, местах питания и местах для проведения досуга. Чтобы определить надобность, не додумывай вопрос пользователя, отвечай только по тому вопросу, что спрашивается. Однако если просят составить маршрут или план на 1 день и более, то отвечай ВСЕГДА [true, true, true].
    В ответе напиши  список из true и false, где на первом месте будет надобность мест проживания, на втором - мест питания (рестораны, кафе и прочее) и на третьем - развлечения (бассейны, парки, музеи и прочее). К примеру, [false, false, true].
    """
    prompt = ChatPromptTemplate.from_template(template)
    result = llm.invoke(prompt.format(question=question))

    try:
        result = result.content
    except AttributeError:
        result = result

    pattern = re.compile(r"\[.*\]")
    match = pattern.search(result)
    #print(match)
    try:
        return json.loads(match[0])
    except:
        return [True,True,True]

In [ ]:
class RawContext:
    def __init__(self, context_name: str = "Unknown", 
                 retriever: VectorStoreRetriever = None, 
                 context_default_output: str = "нет необходимости"):
        self.context_name = context_name
        self.retriever = retriever
        self.context_default_output = context_default_output


def get_contexts_(llm, question: str, raw_contexts: list[RawContext]):
    start_time = time.time()
    categories_bool_list = get_retriver_categories(llm, question)
   # print("Определение необходимых категорий для ретривера: ", time.time()-start_time)
   # print(categories_bool_list)
    start_time = time.time()

    result_context = {
        "question": question,
    }
    for raw_context, is_needed in zip(raw_contexts, categories_bool_list):
        if is_needed:
            extracted_context = (raw_context.retriever | format_docs).invoke(question)
        else:
            extracted_context = raw_context.context_default_output
        result_context[raw_context.context_name] = extracted_context
    
  #  print("Определение контекста из ретривера: ", time.time()-start_time)
    return result_context

# Пример вызова rag_chain
def run_rag_chain(question, prompt, prompt_paths, history="", retr_question="",history_coord_arr=[]):
    # contexts = get_contexts(llm, question)
    context_question = question if retr_question=="" else retr_question
    if ("рядом с новос" in question.lower() or "недалеко с новос" in question.lower()):
        context_question = context_question.lower().replace("новосибирск", "")
        context_question += "например, в Бердске, Кольцово, Колывани"
    contexts = get_contexts_(llm, context_question, raw_contexts=
                             [
                                RawContext(context_name="context_hotels", retriever=hotels_retriever),
                                RawContext(context_name="context_cafes", retriever=cafes_retriever),
                                RawContext(context_name="context_ent", retriever=ent_retriever)
                             ])
    #print(contexts)
    if retr_question != "":
        contexts['question'] = question
    start_time = time.time()
    descr_cafes = []
    descr_hotels = []
    descr_ent = []

    retr_result = contexts

    empty_cat = 0

    for retr in retr_result:
        if retr == 'context_hotels' and retr_result[retr] != 'нет необходимости':
            descr_hotels += retr_result[retr].split(sep='\n\n')
        if retr == 'context_ent' and retr_result[retr] != 'нет необходимости':
            descr_ent += retr_result[retr].split(sep='\n\n')
        if retr == 'context_cafes' and retr_result[retr] != 'нет необходимости':
            descr_cafes += retr_result[retr].split(sep='\n\n')
        if retr_result[retr] == 'нет необходимости':
            empty_cat += 1

    all_descr_df = descr_cafes + descr_hotels + descr_ent
    #print(all_descr_df)
    final_prompt = prompt.format(**(contexts))  # Формируем финальный промпт
    if history != "":
        final_prompt+="\nИстория диалога: " + history
    response = llm.invoke(final_prompt)  # Вызываем LLM с финальным промптом
   # print("Формирование ответа: ", time.time()-start_time)
    start_time = time.time()
   # print(response.content)
    search_text = response.content.lower() + question
    #print(search_text)
    if (("маршрут" not in search_text) and ("план" not in search_text) and ("поездк" not in search_text) and ("путь" not in search_text) and ("пути" not in search_text) and ("путешеств" not in search_text) and ("рядом" not in search_text) or ("недалеко" not in search_text)) or (empty_cat>=2):
        return response.content, []
    #print("Сейчас напишу примерное расстояние между местами...")
    coord_array = get_coordinates_array(all_descr_df)
   # print("Получение координат для мест: ", time.time()-start_time)
    start_time = time.time()

    distances = get_distances(coord_array+history_coord_arr)
   # print("Вычисление расстояний между местами: ", time.time()-start_time)
    start_time = time.time()

    path_contexts = dict()
    path_contexts['distances'] = distances
    path_contexts['prompt'] = response.content

    final_prompt = prompt_paths.format(**path_contexts)  # Формируем финальный промпт
    #print(final_prompt)
    response = llm.invoke(final_prompt)  # Вызываем LLM с финальным промптом

    #contexts['distances'] = distances
    #print(distances)

    try:
        return response.content, coord_array
    except AttributeError:
        return response, coord_array

In [11]:
from typing import Dict, List
from dataclasses import dataclass
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

@dataclass
class ChatMessage:
    role: str  # "user" или "assistant"
    content: str

class DialogManager:
    def __init__(self):
        self.sessions: Dict[str, List[ChatMessage]] = {}
        self.retriever_prompt = (
            "Просто переформулируй последнее сообщение user, чтобы его посыл был понятен досконально без всей остальной истории. Если последнее сообщение не нуждается в переформулировке, то напиши его, какое оно есть. НЕ ВЗДУМАЙ ОТВЕЧАТЬ НА СООБЩЕНИЕ"
        )
        self.contextualize_q_prompt = ChatPromptTemplate.from_messages([
            ("system", self.retriever_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{input}"),
        ])
        self.coord_retr_array = []
        
    def get_session(self, username: str) -> List[ChatMessage]:
        """Возвращает или создает сессию для пользователя"""
        if username not in self.sessions:
            self.sessions[username] = []
        return self.sessions[username]
    
    def add_message(self, username: str, role: str, content: str):
        """Добавляет сообщение в историю чата пользователя"""
        session = self.get_session(username)
        session.append(ChatMessage(role=role, content=content))
    
    def get_chat_history(self, username: str) -> List[ChatMessage]:
        """Возвращает историю чата пользователя"""
        return self.get_session(username)
    
    def clear_history(self, username: str):
        """Очищает историю чата пользователя"""
        self.sessions[username] = []

    def get_user_questions(self, chat_history: List[ChatMessage]) -> List[str]:
        """Возвращает только вопросы пользователя из истории"""
        return [msg.content for msg in chat_history if msg.role == "user"]
    
    def format_chat_history_as_str(self, chat_history: List[ChatMessage]) -> str:
        """Форматирует историю чата в строку"""
        history_str = ""
        for msg in chat_history:
            role = "Пользователь" if msg.role == "user" else "Бот"
            history_str += f"{role}: {msg.content}\n"
        return history_str.strip()
    
    async def generate_response(self, username: str, user_message: str, prompt, prompt_paths) -> str:
        """Генерирует ответ с учетом истории чата"""
        # Добавляем сообщение пользователя в историю
        self.add_message(username, "user", user_message)
        
        # Получаем историю чата
        chat_history = self.get_chat_history(username)
        
        # Получаем только вопросы пользователя для contextualize_q_prompt
        user_questions = self.get_user_questions(chat_history[:-1])  # исключаем текущий вопрос
        messages_for_prompt = [("user", q) for q in user_questions]
        
        # Переформулируем вопрос с учетом истории
        contextualized_question_prompt = self.contextualize_q_prompt.format_messages(
            chat_history=messages_for_prompt,
            input=user_message
        )
        try:
            contextualized_question = llm.invoke(contextualized_question_prompt)
            contextualized_question = contextualized_question.content if hasattr(contextualized_question, 'content') else contextualized_question
        except Exception as e:
            print(f"Ошибка при контекстуализации вопроса: {e}")
            contextualized_question = user_message
        # Форматируем всю историю диалога в строку для RAG
        full_history_str = self.format_chat_history_as_str(chat_history)

        # Генерируем ответ с помощью RAG
        print("Переформулированный вопрос: ", contextualized_question)
        response, coord_array = run_rag_chain(user_message, prompt, prompt_paths, full_history_str, contextualized_question, self.coord_retr_array)
        
        self.coord_retr_array = coord_array

        # Добавляем ответ ассистента в историю
        self.add_message(username, "assistant", response)
        
        return response

In [ ]:
# Инициализация менеджера диалогов
dialog_manager = DialogManager()

# Пример использования
username = "test_user3"  # или любой другой идентификатор

# 2. Диалог с сохранением истории
messages = [
    ("Привет", None),
    ("как дела?", None),
    ("что ты умеешь?", None),
    ("Какие есть парки в Новосибирске?", None),
    ("А какие из них подойдут для детей?", None),
    ("Найди мне кафе рядом с этими местами", None),
    ("А еще?", None),
    ("Где остановиться после развлечений?", None),
    ("спасибо", None)
]

for msg, _ in messages:
    print(f"Пользователь: {msg}")
    response = await dialog_manager.generate_response(
        username=username,
        user_message=msg,
        prompt=prompt,
        prompt_paths=prompt_paths
    )
    print(f"Бот: {response}\n")

Пользователь: Привет
Переформулированный вопрос:  Привет!
<re.Match object; span=(0, 21), match='[false, false, false]'>
[]
Бот: Привет! Как я могу помочь вам сегодня?

Пользователь: как дела?
Переформулированный вопрос:  Привет! Как у тебя дела?
<re.Match object; span=(0, 21), match='[false, false, false]'>
[]
Бот: Бот: У меня всё хорошо, спасибо! Как я могу помочь вам сегодня? Может, хотите составить маршрут по Новосибирской области?

Пользователь: что ты умеешь?
Переформулированный вопрос:  Что ты можешь сделать?
<re.Match object; span=(0, 21), match='[false, false, false]'>
[]
Бот: Я могу помочь вам составить маршруты по Новосибирской области, предлагая интересные места для посещения, рестораны и варианты для ночлега. Если у вас есть конкретные запросы или города, дайте знать!

Пользователь: Какие есть парки в Новосибирске?
Переформулированный вопрос:  В Новосибирске есть несколько парков, среди которых: Центральный парк, парк "Сосновый бор", парк "Берёзовая роща", парк "Караси", а

----------------------------------------------------------------------------------------------------------------------------------------------------------------